<a href="https://colab.research.google.com/github/aruaru0/Foursquare-Location-Matching/blob/main/train_lightgbm.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!mkdir -p ~/.kaggle
!cp /content/drive/MyDrive/datas/kaggle.json  ~/.kaggle/

In [2]:
!pip install lightgbm==3.3.1

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 2.0 MB 5.3 MB/s 
  Attempting uninstall: lightgbm
    Found existing installation: lightgbm 2.2.3
    Uninstalling lightgbm-2.2.3:
      Successfully uninstalled lightgbm-2.2.3


In [3]:
!pip show lightgbm

Name: lightgbm
Version: 3.3.1
Summary: LightGBM Python Package
Home-page: https://github.com/microsoft/LightGBM
Author: None
Author-email: None
License: The MIT License (Microsoft)
Location: /usr/local/lib/python3.7/dist-packages
Requires: scipy, numpy, scikit-learn, wheel
Required-by: 


In [4]:
!pip install kaggle
!apt install unzip

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Reading package lists... Done
Building dependency tree       
Reading state information... Done
unzip is already the newest version (6.0-21ubuntu1.1).
The following package was automatically installed and is no longer required:
  libnvidia-common-460
Use 'apt autoremove' to remove it.
0 upgraded, 0 newly installed, 0 to remove and 45 not upgraded.


In [5]:
!chmod 600 /root/.kaggle/kaggle.json

In [6]:
!mkdir input output

In [7]:
!kaggle competitions download -c foursquare-location-matching

 91% 144M/159M [00:00<00:00, 190MB/s]
100% 159M/159M [00:01<00:00, 145MB/s]


In [8]:
!unzip foursquare-location-matching.zip -d input/foursquare-location-matching

Archive:  foursquare-location-matching.zip
  inflating: input/foursquare-location-matching/pairs.csv  
  inflating: input/foursquare-location-matching/sample_submission.csv  
  inflating: input/foursquare-location-matching/test.csv  
  inflating: input/foursquare-location-matching/train.csv  


In [9]:
#!kaggle kernels output aruaru0/training-data-for-binary-lgb-baseline-0-834 -p ./input/train_data

In [31]:
# !pip uninstall --yes lightgbm && pip install --install-option=--gpu lightgbm
!pip install optuna
!pip install pyarrow
!pip install fastparquet

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 1.5 MB 5.0 MB/s 
     |████████████████████████████████| 140 kB 91.2 MB/s 
     |████████████████████████████████| 1.6 MB 72.1 MB/s 


## lightgbm

In [1]:
import lightgbm as lgb

import numpy as np
import pandas as pd
from sklearn.metrics import accuracy_score
from sklearn.metrics import confusion_matrix
from sklearn.utils.class_weight import compute_sample_weight
import seaborn as sns
import pickle
import gc
import os

In [2]:
class CFG:
    seed = 46
    target = "point_of_interest"
    n_splits = 3

    n_neighbors = 25
    is_debug = False

    model_path = '/content/drive/MyDrive/datas/foursquare'

In [3]:
def reduce_memory(df):
    for col in df.columns:
        col_type = df[col].dtypes
        if col_type != object:
            cmin = df[col].min()
            cmax = df[col].max()
            if str(col_type)[:3] == 'int':
                if cmin > np.iinfo(np.int8).min and cmax < np.iinfo(np.int8).max:
                    df[col] = df[col].astype(np.int8)
                elif cmin > np.iinfo(np.int16).min and cmax < np.iinfo(np.int16).max:
                    df[col] = df[col].astype(np.int16)
                elif cmin > np.iinfo(np.int32).min and cmax < np.iinfo(np.int32).max:
                    df[col] = df[col].astype(np.int32)
                elif cmin > np.iinfo(np.int64).min and cmax < np.iinfo(np.int64).max:
                    df[col] = df[col].astype(np.int64)
            else:
                if cmin > np.finfo(np.float16).min and cmax < np.finfo(np.float16).max:
                    df[col] = df[col].astype(np.float16)
                elif cmin > np.finfo(np.float32).min and cmax < np.finfo(np.float32).max:
                    df[col] = df[col].astype(np.float32)
                else:
                    df[col] = df[col].astype(np.float64)
    return df

In [4]:
sel = ""

In [5]:
df1 = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data1.csv'))
df2 = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data2.csv'))
df3 = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data3.csv'))
df4 = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data4.csv'))
df5 = reduce_memory(pd.read_csv(f'/content/drive/MyDrive/datas/foursquare/traindata{sel}/train_data5.csv'))

In [6]:
from sklearn.model_selection import GroupKFold, KFold, StratifiedKFold
from tqdm import tqdm

def add_fold(df) :
  kf = StratifiedKFold(n_splits=CFG.n_splits, shuffle=True, random_state=CFG.seed)
  for i, (trn_idx, val_idx) in tqdm(enumerate(kf.split(df, df["label"], df["label"]))):
      df.loc[val_idx, "fold"] = i
  return df


In [7]:
df1 = add_fold(df1)
df2 = add_fold(df2)
df3 = add_fold(df3)
df4 = add_fold(df4)
df5 = add_fold(df5)

3it [00:00,  4.76it/s]
3it [00:00,  4.75it/s]
3it [00:00,  4.82it/s]
3it [00:00,  4.82it/s]
3it [00:00,  4.81it/s]


In [8]:
train = pd.concat([df1,df2,df3,df4,df5])
train = train.reset_index(drop=True)

In [9]:
train.head()

In [10]:
del df1, df2, df3, df4, df5
gc.collect()

111

In [11]:
train['kdist_diff'] = (train['kdist'] - train['kdist_country']) /\
                            train['kdist_country']
train['kneighbors_mean'] = train[['kneighbors', 'kneighbors_country']].mean(axis = 1)

In [12]:
train = reduce_memory(train)

In [17]:
gc.collect()

152

In [18]:
# str | Noneって書きたいところだけど3.7系でもいけるようにOptionalを使う
from typing import Optional
from collections import OrderedDict
from lightgbm.callback import CallbackEnv
from tqdm.auto import tqdm

class LgbmProgressBarCallback:
    description: Optional[str]
    pbar: tqdm

    def __init__(self, description: Optional[str] = None):
        self.description = description
        self.pbar = tqdm()

    def __call__(self, env: CallbackEnv):

        # 初回だけProgressBarを初期化する
        is_first_iteration: bool = env.iteration == env.begin_iteration

        if is_first_iteration:
            total: int = env.end_iteration - env.begin_iteration
            self.pbar.reset(total=total)
            self.pbar.set_description(self.description, refresh=False)

        # valid_setsの評価結果を更新
        if len(env.evaluation_result_list) > 0:
            # OrderedDictにしないと表示順がバラバラになって若干見にくい
            postfix = OrderedDict(
                [
                    (f"{entry[0]}:{entry[1]}", str(entry[2]))
                    for entry in env.evaluation_result_list
                ]
            )
            self.pbar.set_postfix(ordered_dict=postfix, refresh=False)

        # 進捗を1進める
        self.pbar.update(1)
        self.pbar.refresh()


In [19]:
def fit_lgbm(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = np.zeros((len(y), n_class), dtype=np.float64)
  
  for i in tqdm(range(CFG.n_splits)):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]

    if model_dir is None:
      # 学習
      model = lgb.LGBMClassifier(**params)
      model.fit(
          X_train, y_train, 
          eval_set=[(X_valid, y_valid)],  
          early_stopping_rounds=es_rounds, 
          eval_metric='logloss',  
          verbose=-1,
          # verbose=50,
          callbacks=[LgbmProgressBarCallback(description="Model A"),],
          )
    else: 
      # 既に学習している場合は、読み込む（トレーニングをしない）
      with open(f'{model_dir}/lgbm_fold{i}.pkl', 'rb') as f:
          model = pickle.load(f)

    # validデータに対して予測し、結果を格納    
    pred = model.predict_proba(X_valid)
    oof[val_idx] = pred
    models.append(model)
    
    # 学習した結果をファイルに保存（！！！colabの場合は、、google driveに保存するように変更しておくしておく！！！）
    #file = f'lgbm_fold{i}.pkl'
    # file = f'{CFG.model_path}/lgbm_fold{i}.pkl'
    # pickle.dump(model, open(file, 'wb'))
    print()
    model.booster_.save_model(f'{CFG.model_path}/lgbm_fold{i}.txt')

  cv = (oof.argmax(axis=-1) == y).mean()
  print(f"CV-accuracy: {cv}")

  return oof, models


In [20]:
import optuna.integration.lightgbm as lgbopt
import lightgbm as lgb

In [21]:
def sample_scheduler_func(current_lr, eval_history, best_round, is_higher_better):
    """次のラウンドで用いる学習率を決定するための関数 (この中身を好きに改造する)

    :param current_lr: 現在の学習率 (指定されていない場合の初期値は None)
    :param eval_history: 検証用データに対する評価指標の履歴
    :param best_round: 現状で最も評価指標の良かったラウンド数
    :param is_higher_better: 高い方が性能指標として優れているか否か
    :return: 次のラウンドで用いる学習率

    NOTE: 学習を打ち切りたいときには callback.EarlyStopException を上げる
    """
    # 学習率が設定されていない場合のデフォルト
    #current_lr = current_lr or 0.05


    # 試しに 20 ラウンド毎に学習率を半分にしてみる
    # if len(eval_history) % 20 == 0:
    #     current_lr /= 2

    # # 小さすぎるとほとんど学習が進まないので下限も用意する
    # min_threshold = 0.001
    # current_lr = max(min_threshold, current_lr)
    n = len(eval_history) 
    
    if n == 100 : # 1.0 -> 0.5
      current_lr *= 0.5
    elif n == 500 : # 0.5 -> 0.2 
      current_lr *= 0.4
    
    return current_lr


class LrSchedulingCallback(object):
    """ラウンドごとの学習率を動的に制御するためのコールバック"""

    def __init__(self, strategy_func):
        # 学習率を決定するための関数
        self.scheduler_func = strategy_func
        # 検証用データに対する評価指標の履歴
        self.eval_metric_history = []

    def __call__(self, env):
        # 現在の学習率を取得する
        current_lr = env.params.get('learning_rate')

        # 検証用データに対する評価結果を取り出す (先頭の評価指標)
        first_eval_result = env.evaluation_result_list[0]
        # スコア
        metric_score = first_eval_result[2]
        # 評価指標は大きい方が優れているか否か
        is_higher_better = first_eval_result[3]

        # 評価指標の履歴を更新する
        self.eval_metric_history.append(metric_score)
        # 現状で最も優れたラウンド数を計算する
        best_round_find_func = np.argmax if is_higher_better else np.argmin
        best_round = best_round_find_func(self.eval_metric_history)

        # 新しい学習率を計算する
        new_lr = self.scheduler_func(current_lr=current_lr,
                                     eval_history=self.eval_metric_history,
                                     best_round=best_round,
                                     is_higher_better=is_higher_better)

        # 次のラウンドで使う学習率を更新する
        update_params = {
            'learning_rate': new_lr,
        }
        env.model.reset_parameter(update_params)
        env.params.update(update_params)

    @property
    def before_iteration(self):
        # コールバックは各イテレーションの後に実行する
        return False

In [22]:
def fit_lgbm_optuna(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = []
  
  for i in range(CFG.n_splits):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]


    train_data = lgb.Dataset(
        data=X_train, 
        label=y_train, 
        # weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
    )
    validation_data = lgb.Dataset(
        data=X_valid, 
        label=y_valid, 
        # weight=np.ones(len(X_valid)).astype('float32'),
    )

    model = lgbopt.train(
      params=params,
      train_set=train_data, 
      num_boost_round=300, 
      early_stopping_rounds=20,
      valid_sets=[validation_data],
      verbose_eval=False,
      callbacks=[LgbmProgressBarCallback(description="Model A"),
                 ],
    )
    print("best_params " , model.params)
    model.save_model(f'{CFG.model_path}/lgbm_fold{i}.txt', num_iteration=model.best_iteration)


    # validデータに対して予測し、結果を格納 
    pred = model.predict(X_valid, num_iteration=model.best_iteration)
    pred = pred.round(0)
    accuracy = accuracy_score(pred, y_valid) 
    print()
    print(accuracy)
    oof.append(pred)
    models.append(model)
    break

  return oof, models


In [23]:

def fit_lgbm2(X, y, params=None, es_rounds=20, seed=42, N_SPLITS=5, 
             n_class=None, model_dir=None, folds=None):
  models = []
  oof = []
  
  for i in range(CFG.n_splits):
    print(f"== fold {i} ==")
    trn_idx = folds!=i
    val_idx = folds==i
    # train, validデータに分割
    X_train, y_train = X[trn_idx], y.iloc[trn_idx]
    X_valid, y_valid = X.iloc[val_idx], y.iloc[val_idx]


    train_data = lgb.Dataset(
        data=X_train, 
        label=y_train, 
        # weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
    )
    validation_data = lgb.Dataset(
        data=X_valid, 
        label=y_valid, 
        # weight=np.ones(len(X_valid)).astype('float32'),
    )

    model = lgb.train(
      params=params,
      train_set=train_data, 
      num_boost_round=500, 
      early_stopping_rounds=20,
      valid_sets=[validation_data],
      verbose_eval=False,
      callbacks=[LgbmProgressBarCallback(description="Model A"),
                 #LrSchedulingCallback(strategy_func=sample_scheduler_func),
                ],
    )
    # print("best_params " , model.params)
    model.save_model(f'{CFG.model_path}/lgbm_fold{i}.txt', num_iteration=model.best_iteration)


    # validデータに対して予測し、結果を格納 
    pred = model.predict(X_valid, num_iteration=model.best_iteration)
    pred = pred.round(0)
    accuracy = accuracy_score(pred, y_valid) 
    print()
    print(accuracy)
    oof.append(pred)
    models.append(model)

  return oof, models


In [24]:
features = ['kdist', 'kneighbors', 'kdist_country',
        'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
        'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
        'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
        'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
        'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
        'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
        'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
        'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
        'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
        'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
        'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
        'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
        'categories_jaro', 'categories_lcs', 'categories_len_diff',
        'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
        'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
        'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
        'country_nlcs']

In [25]:
TRAIN_FEATURES = ['kdist',
                'kneighbors',
                'kdist_country',
                'kneighbors_country',
                'latdiff',
                'londiff',
                'manhattan',
                'euclidean',
                'haversine',
                'name_sim',
                'name_gesh',
                'name_leven',
                'name_jaro',
                'name_lcs',
                'name_len_diff',
                'name_nleven',
                'name_nlcsk',
                'name_nlcs',
                'address_sim',
                'address_gesh',
                'address_leven',
                'address_jaro',
                'address_lcs',
                'address_len_diff',
                'address_nleven',
                'address_nlcsk',
                'address_nlcs',
                'city_gesh',
                'city_leven',
                'city_jaro',
                'city_lcs',
                'city_len_diff',
                'city_nleven',
                'city_nlcsk',
                'city_nlcs',
                'state_sim',
                'state_gesh',
                'state_leven',
                'state_jaro',
                'state_lcs',
                'state_len_diff',
                'state_nleven',
                'state_nlcsk',
                'state_nlcs',
                'zip_gesh',
                'zip_leven',
                'zip_jaro',
                'zip_lcs',
                'url_sim',
                'url_gesh',
                'url_leven',
                'url_jaro',
                'url_lcs',
                'url_len_diff',
                'url_nleven',
                'url_nlcsk',
                'url_nlcs',
                'phone_gesh',
                'phone_leven',
                'phone_jaro',
                'phone_lcs',
                'categories_sim',
                'categories_gesh',
                'categories_leven',
                'categories_jaro',
                'categories_lcs',
                'categories_len_diff',
                'categories_nleven',
                'categories_nlcsk',
                'categories_nlcs',
                'country_sim',
                'country_gesh',
                'country_leven',
                'country_nleven',
                'country_jaro',
                'country_lcs', 'country_len_diff', 'country_nlcsk',
                'country_nlcs',
                'kdist_diff',
                'kneighbors_mean',]

In [26]:
features = ['kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
       'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
       'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
       'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
       'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
       'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
       'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
       'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
       'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
       'categories_jaro', 'categories_lcs', 'categories_len_diff',
       'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
       'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
       'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
       'country_nlcs', 'kdist_diff', 'kneighbors_mean']

In [27]:
for e in train.columns : 
  if e not in features : print(e)

id
match_id
label
fold


In [28]:
train.columns

Index(['id', 'match_id', 'kdist', 'kneighbors', 'kdist_country',
       'kneighbors_country', 'label', 'latdiff', 'londiff', 'manhattan',
       'euclidean', 'haversine', 'name_sim', 'name_gesh', 'name_leven',
       'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
       'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
       'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
       'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
       'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
       'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
       'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
       'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
       'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
       'phone_lcs', 'categories_sim', 

In [29]:
# params = {
#     'objective': "binary",
#     'learning_rate': 0.2,
#     'reg_alpha': 0.1,
#     'reg_lambda': 0.1,
#     'random_state': 42,

#     'max_depth': 7,   
#     'num_leaves': 35, 
#     'n_estimators': 1000000, 
#     "colsample_bytree": 0.9,
# }

# params = {
#   'task': 'train',
#   'boosting_type': 'gbdt',
#   'objective': 'binary',
#   'metric': 'binary_logloss',
#   'num_leaves': 48,
#   'min_data_in_leaf': 200,
#   'max_depth': 6,

#   'learning_rate': 0.2,
#   'reg_alpha': 0.1,
#   'reg_lambda': 0.1,
#   "colsample_bytree": 0.7,

#   # 'n_estimators': 5000,
#   'random_state': 42,
# }

# params = {
#   'task': 'train',
#   'boosting_type': 'gbdt',
#   'objective': "binary",
#   'learning_rate': 0.2,
#   'random_state': 42,
#   'n_estimators': 100,
# }

# params = {
#     'objective': "binary",
#     'learning_rate': 0.2,
#     'reg_alpha': 0.1,
#     'reg_lambda': 0.1,
#     'random_state': 42,
#     'min_data_in_leaf': 20,

#     'max_depth': 7,   
#     'num_leaves': 200, 
#     # 'n_estimators': 1000000, 
#     'n_estimators': 10000, 
#     "colsample_bytree": 0.9,
#     "verbose": -1,
# }
# params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 
#           'num_leaves': 61, 'min_data_in_leaf': 200, 'max_depth': 6, 'learning_rate': 0.2, 'random_state': 42, 
#           'feature_pre_filter': False, 'feature_fraction': 0.88, 'lambda_l1': 5.5168523748814025, 'lambda_l2': 0.029477410051213494, 
#           'bagging_fraction': 0.9846184137245049, 'bagging_freq': 7, 'min_child_samples': 25}

# params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss',
#           'learning_rate': 0.05, 
#           'num_leaves': 200,
#           'max_depth': -1, 
#           'min_data_in_leaf': 20,  
#           'random_state': 42, 

#           'lambda_l1': 1,
#           'max_bin': 255,
#           'feature_pre_filter': True, 
#           'feature_fraction': 0.75, 
#           'bagging_fraction': 0.80, 
#           'bagging_freq': 10, 

#           "verbose": -1,
#           }

params = {'task': 'train', 'boosting_type': 'gbdt', 'objective': 'binary', 'metric': 'binary_logloss', 
          'learning_rate': 0.05, 'num_leaves': 256, 'max_depth': -1, 'min_data_in_leaf': 20, 
          'random_state': 42, 
          'lambda_l1': 0.7964782150381488,
          'max_bin': 255, 'feature_pre_filter': False, 
          'feature_fraction': 0.4,
          'bagging_fraction': 0.9277418926611961, 'bagging_freq': 5,
          'verbose': -1, 'lambda_l2': 0.7677209353759652, 'min_child_samples': 10,
          #'is_unbalance': True,
          }


oof, models = fit_lgbm2(train[features], train["label"].astype(int), 
                      params=params, 
                      n_class=int(train["label"].max() + 1), 
                      N_SPLITS=CFG.n_splits, 
                      folds=train["fold"].values)

== fold 0 ==


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=10 will be ignored. Current value: min_data_in_leaf=20

0.9871072506072454
== fold 1 ==


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=10 will be ignored. Current value: min_data_in_leaf=20

0.9873041166255913
== fold 2 ==


0it [00:00, ?it/s]

/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:181: UserWarning: 'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. Pass 'early_stopping()' callback via 'callbacks' argument instead.
  _log_warning("'early_stopping_rounds' argument is deprecated and will be removed in a future release of LightGBM. "
/usr/local/lib/python3.7/dist-packages/lightgbm/engine.py:239: UserWarning: 'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. Pass 'log_evaluation()' callback via 'callbacks' argument instead.
  _log_warning("'verbose_eval' argument is deprecated and will be removed in a future release of LightGBM. "


[LightGBM] [Warning] min_data_in_leaf is set=20, min_child_samples=10 will be ignored. Current value: min_data_in_leaf=20

0.9872967878063638


In [ ]:
# 0.9832166157227484
# 0.986433063288391
#0.9879528738117864 0.9877047494263559

In [ ]:
# import gc
# del df1, df2, df3, df4, df5 
# gc.collect()

In [ ]:
# def set_datas(train, valid, test) :
#   df = pd.concat(train)
#   x_train = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_train = df['label']

#   df = pd.concat(valid)
#   x_valid = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_valid = df['label']

#   df = pd.concat(test)
#   x_test = df[['kdist', 'kneighbors', 'kdist_country',
#         'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#         'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#         'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#         'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#         'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#         'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#         'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#         'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#         'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#         'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#         'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#         'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#         'categories_jaro', 'categories_lcs', 'categories_len_diff',
#         'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#         'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#         'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#         'country_nlcs']]
#   y_test = df['label']
#   del df
#   return x_train, y_train, x_valid, y_valid, x_test, y_test

In [ ]:
# def train(train, valid, test, outfile): 
#   x_train, y_train, x_valid, y_valid, x_test, y_test = set_datas(train, valid, test)

#   train_data = lgb.Dataset(
#       data=x_train, 
#       label=y_train, 
#       weight=compute_sample_weight(class_weight='balanced', y=y_train).astype('float32'),
#   )
#   validation_data = lgb.Dataset(
#       data=x_valid, 
#       label=y_valid, 
#       weight=np.ones(len(x_valid)).astype('float32'),
#   )
#   params = {
#     'task': 'train',
#     'boosting_type': 'gbdt',
#     'objective': 'binary',
#     'metric': 'binary_logloss',
#     'num_leaves': 64,
#     'min_data_in_leaf': 20,
#     'max_depth': 7,
#     'verbose': 0,

#     'learning_rate': 0.2,
#     'random_state': 42,
#   }
#   # params = {
#   #   'objective': "logloss",
#   #   'learning_rate': 0.2,
#   #   'reg_alpha': 0.1,
#   #   'reg_lambda': 0.1,
#   #   'random_state': 42,

#   #   'max_depth': 7,   
#   #   'num_leaves': 35, 
#   #   'n_estimators': 1000000, 
#   #   "colsample_bytree": 0.9,
#   # }

#   bst = lgb.train(
#     params=params, 
#     train_set=train_data, 
#     num_boost_round=100, 
#     early_stopping_rounds=20,
#     valid_sets=[validation_data],
#   )
#   bst.save_model(outfile, num_iteration=bst.best_iteration)

#   y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
#   y_pred = y_pred.round(0)
#   accuracy = accuracy_score(y_pred, y_test)
#   print(f"accuracy score: {accuracy:0.4f}")
#   # accuracy score: 0.9825
#   cm = confusion_matrix(y_test, y_pred)
#   cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                   index=['Predict Positive:1', 'Predict Negative:0'])

#   sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')
#   del x_train, y_train, x_valid, y_valid, x_test, y_test, bst, y_pred, cm_matrix

In [ ]:
# train([df1,df2,df3,df4], [df5], [df5], '/content/drive/MyDrive/datas/foursquare/model1.txt')
# gc.collect()

In [ ]:
# train([df2,df3,df4], [df5], [df1], '/content/drive/MyDrive/datas/foursquare/model2.txt')
# gc.collect()

In [ ]:
# train([df3,df4,df5], [df1], [df2], '/content/drive/MyDrive/datas/foursquare/model3.txt')
# gc.collect()

In [ ]:
# train([df4,df5,df1], [df2], [df3], '/content/drive/MyDrive/datas/foursquare/model4.txt')
# gc.collect()

In [ ]:
# train([df5,df1,df2], [df3], [df4], '/content/drive/MyDrive/datas/foursquare/model5.txt')
# gc.collect()

In [ ]:
# df = df1
# x_test = df[['kdist', 'kneighbors', 'kdist_country',
#       'kneighbors_country', 'name_sim', 'name_gesh', 'name_leven',
#       'name_jaro', 'name_lcs', 'name_len_diff', 'name_nleven', 'name_nlcsk',
#       'name_nlcs', 'address_sim', 'address_gesh', 'address_leven',
#       'address_jaro', 'address_lcs', 'address_len_diff', 'address_nleven',
#       'address_nlcsk', 'address_nlcs', 'city_gesh', 'city_leven', 'city_jaro',
#       'city_lcs', 'city_len_diff', 'city_nleven', 'city_nlcsk', 'city_nlcs',
#       'state_sim', 'state_gesh', 'state_leven', 'state_jaro', 'state_lcs',
#       'state_len_diff', 'state_nleven', 'state_nlcsk', 'state_nlcs',
#       'zip_gesh', 'zip_leven', 'zip_jaro', 'zip_lcs', 'url_sim', 'url_gesh',
#       'url_leven', 'url_jaro', 'url_lcs', 'url_len_diff', 'url_nleven',
#       'url_nlcsk', 'url_nlcs', 'phone_gesh', 'phone_leven', 'phone_jaro',
#       'phone_lcs', 'categories_sim', 'categories_gesh', 'categories_leven',
#       'categories_jaro', 'categories_lcs', 'categories_len_diff',
#       'categories_nleven', 'categories_nlcsk', 'categories_nlcs',
#       'country_sim', 'country_gesh', 'country_leven', 'country_jaro',
#       'country_lcs', 'country_len_diff', 'country_nleven', 'country_nlcsk',
#       'country_nlcs']]
# y_test = df['label']

In [ ]:
# bst = lgb.Booster(model_file = '/content/drive/MyDrive/datas/foursquare/improved_lgb_baseline.lgb')

# y_pred = bst.predict(x_test, num_iteration=bst.best_iteration)
# y_pred = y_pred.round(0)
# accuracy = accuracy_score(y_pred, y_test)
# print(f"accuracy score: {accuracy:0.4f}")
# # accuracy score: 0.9825
# cm = confusion_matrix(y_test, y_pred)
# cm_matrix = pd.DataFrame(data=cm, columns=['Actual Positive:1', 'Actual Negative:0'], 
#                                 index=['Predict Positive:1', 'Predict Negative:0'])

# sns.heatmap(cm_matrix, annot=True, fmt='d', cmap='YlGnBu')